In [1]:
# ! pip install graphviz
# ! pip install hiddenlayer 
# ! git clone --branch feature/prerak2 https://github.com/prerakmody/CS4180-DL
# ! wget -P /content/CS4180-DL/data/weights https://pjreddie.com/media/files/yolov2-voc.weights

%load_ext autoreload
%autoreload 2

import os
import sys
import torch
import hiddenlayer as hl

DIR_MAIN      = os.path.abspath('../../../CS4180-DL')
sys.path.append(DIR_MAIN)
from src.nets import *

 - Online Runtime


In [2]:
MODEL_CFG     = os.path.join(DIR_MAIN, 'data/cfg/github_pjreddie/yolov2-voc.cfg')
MODEL_WEIGHTS = os.path.join(DIR_MAIN, 'data/weights/yolov2-voc.weights')
model         = getYOLOv2(MODEL_CFG, MODEL_WEIGHTS)

  -- [DEBUG] Non-BN Block :  {'type': 'convolutional', 'batch_normalize': 0, 'size': '1', 'stride': '1', 'pad': '1', 'filters': '125', 'activation': 'linear'}
  -- [DEBUG][Darknet] self.anchors :  [1.3221, 1.73145, 3.19275, 4.00944, 5.05587, 8.09892, 9.47112, 4.84053, 11.2364, 10.0071]
  -- [DEBUG][Darknet] self.num_anchors :  5
  -- [DEBUG][Darknet] self.anchor_step :  2.0
  -- [DEBUG][Darknet] self.num_classes :  20
  -- [DEBUG][Darknet] self.loss :  RegionLoss()


In [15]:
transforms = [
            # Fold Conv, BN, RELU layers into one
            hl.transforms.Fold("Conv > BatchNorm > LeakyRelu", "ConvBnRelu"),
            hl.transforms.Fold("ConvBnRelu > MaxPool", "ConvBnReluMax"),
            hl.transforms.Fold("Constant > Reshape", "ConstantReshape"),
            hl.transforms.Fold("ConstantReshape > Transpose", "Re-Tra")
]
transforms = [
            # Fold Conv, BN, RELU layers into one
            hl.transforms.Fold("Conv > BatchNorm > LeakyRelu", "ConvBnRelu"),
            hl.transforms.Fold("ConvBnRelu > MaxPool", "ConvBnReluMax"),
            hl.transforms.Fold("Constant > Reshape > Transpose", "ConstantReshape")
]


g = hl.build_graph(model, torch.zeros([1, 3, 416, 416]).cuda(), transforms=transforms)
g.save(os.path.join("pytorch_yolov2.pdf"))
# hl.build_graph(model, torch.zeros([1, 3, 416, 416]).cuda(), transforms=transforms)


(dot.exe:5476): Pango-WARNING **: couldn't load font "Times Not-Rotated 10", falling back to "Sans Not-Rotated 10", expect ugly output.
